# Visualization notebook

This notebook is meant for visualizing stuff and testing code. 


## Imports and functions

In [4]:
import utils
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import plotly.express as px
from pathlib import Path
import numpy as np
import pandas as pd
import warnings
import spacy
import json
from nltk import bigrams
from pathlib import Path
from collections import Counter
import re
warnings.filterwarnings("ignore", category=FutureWarning)
%matplotlib inline

# MUD: 1,071,477 authors

### Plotting by bin

In [40]:
bin_nums = "1 2 3 4 5 6 7 8".split()

dev_bin_avg_docs = []




# manually recorded
HALF_FEATS_ACCS = [0.0, 0.02857142857142857, 0.08571428571428572, 0.11428571428571428, 0.22857142857142856, 0.22857142857142856, 0.22857142857142856, 0.4]
ALL_FEATS_ACCS = [0.0, 0.05714285714285714, 0.11428571428571428, 0.17142857142857143, 0.2, 0.2, 0.2857142857142857, 0.5428571428571428]


 

df = pd.DataFrame.from_dict({"half_features": HALF_FEATS_ACCS,
                             "all_features" : ALL_FEATS_ACCS,
                             "bin":bin_nums})


fig = px.line(df, x="bin", y=df.columns[0:2], template="plotly_dark", markers=True,)
fig.update_layout(title_text="Accuracy per bin", title_x=0.5)



## Backups

### DOC-SORTED DEV BINS (metric=euclid)

In [ ]:



{
  "results": [
    {
      "datetime": "2023-01-09 17:19:32.731050",
      "acc": 0.4857142857142857,
      "config": [
        "pos_unigrams",
        "pos_bigrams",
        "func_words",
        "punc",
        "letters",
        "common_emojis",
        "doc_vector",
        "doc_stats",
        "dep_labels",
        "mixed_bigrams"
      ],
      "vector_length": "707",
      "k": "7"
    },
    {
      "datetime": "2023-01-09 17:19:47.934214",
      "acc": 0.2571428571428571,
      "config": [
        "pos_unigrams",
        "pos_bigrams",
        "func_words",
        "punc",
        "letters",
        "common_emojis",
        "doc_vector",
        "doc_stats",
        "dep_labels",
        "mixed_bigrams"
      ],
      "vector_length": "707",
      "k": "7"
    },
    {
      "datetime": "2023-01-09 17:20:03.183383",
      "acc": 0.22857142857142856,
      "config": [
        "pos_unigrams",
        "pos_bigrams",
        "func_words",
        "punc",
        "letters",
        "common_emojis",
        "doc_vector",
        "doc_stats",
        "dep_labels",
        "mixed_bigrams"
      ],
      "vector_length": "707",
      "k": "7"
    },
    {
      "datetime": "2023-01-09 17:20:18.623211",
      "acc": 0.14285714285714285,
      "config": [
        "pos_unigrams",
        "pos_bigrams",
        "func_words",
        "punc",
        "letters",
        "common_emojis",
        "doc_vector",
        "doc_stats",
        "dep_labels",
        "mixed_bigrams"
      ],
      "vector_length": "707",
      "k": "7"
    },
    {
      "datetime": "2023-01-09 17:20:34.516043",
      "acc": 0.14285714285714285,
      "config": [
        "pos_unigrams",
        "pos_bigrams",
        "func_words",
        "punc",
        "letters",
        "common_emojis",
        "doc_vector",
        "doc_stats",
        "dep_labels",
        "mixed_bigrams"
      ],
      "vector_length": "707",
      "k": "7"
    },
    {
      "datetime": "2023-01-09 17:20:50.119132",
      "acc": 0.22857142857142856,
      "config": [
        "pos_unigrams",
        "pos_bigrams",
        "func_words",
        "punc",
        "letters",
        "common_emojis",
        "doc_vector",
        "doc_stats",
        "dep_labels",
        "mixed_bigrams"
      ],
      "vector_length": "707",
      "k": "7"
    },
    {
      "datetime": "2023-01-09 17:21:05.201044",
      "acc": 0.05714285714285714,
      "config": [
        "pos_unigrams",
        "pos_bigrams",
        "func_words",
        "punc",
        "letters",
        "common_emojis",
        "doc_vector",
        "doc_stats",
        "dep_labels",
        "mixed_bigrams"
      ],
      "vector_length": "707",
      "k": "7"
    },
    {
      "datetime": "2023-01-09 17:21:20.507916",
      "acc": 0.0,
      "config": [
        "pos_unigrams",
        "pos_bigrams",
        "func_words",
        "punc",
        "letters",
        "common_emojis",
        "doc_vector",
        "doc_stats",
        "dep_labels",
        "mixed_bigrams"
      ],
      "vector_length": "707",
      "k": "7"
    }
  ]
}

### DOC SORTED DEV BINS (metric=cosine)

In [ ]:
{
  "results": [
    {
      "datetime": "2023-01-17 13:15:39.774486",
      "acc": 0.5428571428571428,
      "vector_length": "707",
      "k": "7",
      "Metric": "cosine",
      "config": [
        "pos_unigrams",
        "pos_bigrams",
        "func_words",
        "punc",
        "letters",
        "common_emojis",
        "doc_vector",
        "doc_stats",
        "dep_labels",
        "mixed_bigrams"
      ]
    },
    {
      "datetime": "2023-01-17 13:15:54.863078",
      "acc": 0.17142857142857143,
      "vector_length": "707",
      "k": "7",
      "Metric": "cosine",
      "config": [
        "pos_unigrams",
        "pos_bigrams",
        "func_words",
        "punc",
        "letters",
        "common_emojis",
        "doc_vector",
        "doc_stats",
        "dep_labels",
        "mixed_bigrams"
      ]
    },
    {
      "datetime": "2023-01-17 13:16:10.055100",
      "acc": 0.2,
      "vector_length": "707",
      "k": "7",
      "Metric": "cosine",
      "config": [
        "pos_unigrams",
        "pos_bigrams",
        "func_words",
        "punc",
        "letters",
        "common_emojis",
        "doc_vector",
        "doc_stats",
        "dep_labels",
        "mixed_bigrams"
      ]
    },
    {
      "datetime": "2023-01-17 13:16:25.346764",
      "acc": 0.2,
      "vector_length": "707",
      "k": "7",
      "Metric": "cosine",
      "config": [
        "pos_unigrams",
        "pos_bigrams",
        "func_words",
        "punc",
        "letters",
        "common_emojis",
        "doc_vector",
        "doc_stats",
        "dep_labels",
        "mixed_bigrams"
      ]
    },
    {
      "datetime": "2023-01-17 13:16:40.599021",
      "acc": 0.05714285714285714,
      "vector_length": "707",
      "k": "7",
      "Metric": "cosine",
      "config": [
        "pos_unigrams",
        "pos_bigrams",
        "func_words",
        "punc",
        "letters",
        "common_emojis",
        "doc_vector",
        "doc_stats",
        "dep_labels",
        "mixed_bigrams"
      ]
    },
    {
      "datetime": "2023-01-17 13:16:55.796938",
      "acc": 0.2857142857142857,
      "vector_length": "707",
      "k": "7",
      "Metric": "cosine",
      "config": [
        "pos_unigrams",
        "pos_bigrams",
        "func_words",
        "punc",
        "letters",
        "common_emojis",
        "doc_vector",
        "doc_stats",
        "dep_labels",
        "mixed_bigrams"
      ]
    },
    {
      "datetime": "2023-01-17 13:17:11.070055",
      "acc": 0.11428571428571428,
      "vector_length": "707",
      "k": "7",
      "Metric": "cosine",
      "config": [
        "pos_unigrams",
        "pos_bigrams",
        "func_words",
        "punc",
        "letters",
        "common_emojis",
        "doc_vector",
        "doc_stats",
        "dep_labels",
        "mixed_bigrams"
      ]
    },
    {
      "datetime": "2023-01-17 13:17:26.285711",
      "acc": 0.0,
      "vector_length": "707",
      "k": "7",
      "Metric": "cosine",
      "config": [
        "pos_unigrams",
        "pos_bigrams",
        "func_words",
        "punc",
        "letters",
        "common_emojis",
        "doc_vector",
        "doc_stats",
        "dep_labels",
        "mixed_bigrams"
      ]
    }
  ]
}